In [ ]:
import numpy as np
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value # only leaf nodes have value
class DecisionTree():
    def __init__(self):
        self.root=None
    def _entropy(self,y):
        classes,count=np.unique(y,return_counts=True)
        prob=count/len(y)
        return -np.sum(prob*np.log2(prob+1e-9)) #avoids taking log(0), which causes errors.
    def _split(self ,X,y,feature_index,threshold):
        leftmask=X[:,feature_index]<=threshold
        rightmask=~leftmask
        return X[leftmask],y[leftmask],X[rightmask],y[rightmask]
    def best_split(self,X,y):
        best_gain=-1
        best_feature=None
        best_threshold=None
        parententropy=self._entropy(y)
        n_features=X.shape[1]
        for feature_index in range(n_features):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                X_left, y_left, X_right, y_right = self._split(X, y, feature_index, threshold)
                if len(y_left) == 0 or len(y_right) == 0:
                    continue
                left_entropy = self._entropy(y_left)
                right_entropy = self._entropy(y_right)
                weighted_entropy = (len(y_left) / len(y)) * left_entropy + (len(y_right) / len(y)) * right_entropy
                info_gain = parententropy - weighted_entropy
                if info_gain > best_gain:
                    best_gain = info_gain
                    best_feature = feature_index
                    best_threshold = threshold
        return best_feature, best_threshold
    def _build_tree(self,X,y,depth=0,max_depth=3):
        # If all labels are the same or max depth is reached, return a leaf node
        if len(set(y)) == 1 or depth >= max_depth:
            leaf_value = max(set(y), key=list(y).count)
            return Node(value=leaf_value)
        feature, threshold = self.best_split(X, y)
        if feature is None:  #if we dont get nice info gain it is better to stop spliting their so return the node
            leaf_value = max(set(y), key=list(y).count)
            return Node(value=leaf_value)
        X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)
        left_child = self._build_tree(X_left, y_left, depth + 1, max_depth)
        right_child = self._build_tree(X_right, y_right, depth + 1, max_depth)

        return Node(feature, threshold, left_child, right_child)
    def predict_sample(self,node, sample):
        if node.value is not None:
            return node.value
        if sample[node.feature] <= node.threshold:
            return self.predict_sample(node.left, sample)
        else:
            return self.predict_sample(node.right, sample)
    def predict(self, X):
        return [self.predict_sample(self.root, sample) for sample in X]
    def fit(self, X, y, max_depth=3):
        self.root = self._build_tree(X, y, 0, max_depth)

In [21]:
X = np.array([[1, 1], [2, 1], [3, 2], [6, 5], [7, 8], [8, 9]])
y = np.array([0, 0, 0, 1, 1, 1])

In [22]:
tree = DecisionTree()
tree.fit(X, y, max_depth=3)

print("Predictions:", tree.predict(X))

Predictions: [np.int64(0), np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1)]


In [23]:
# Features: [Age, Salary]
X = np.array([
    [22, 25],
    [25, 30],
    [47, 60],
    [52, 80],
    [46, 50],
    [56, 90],
    [28, 40],
    [30, 60]
])

# Target: 0 = No, 1 = Yes
y = np.array([0, 0, 1, 1, 1, 1, 0, 0])

In [24]:
tree = DecisionTree()
tree.fit(X, y, max_depth=3)

predictions = tree.predict(X)
print("Predictions:", predictions)
print("Actual:     ", y.tolist())

Predictions: [np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0)]
Actual:      [0, 0, 1, 1, 1, 1, 0, 0]


In classic decision tree algorithms (like ID3, C4.5, or CART):

- Each split is made independently.

- At every node, we search all features and all thresholds again.

- If the same feature is still the best choice, we use it again.

if you want to prevent using the same feature again (e.g., for some educational or constrained reason), you'd need to:

- Track used features, and

- Remove them from the candidate list in best_split().




RANDOM FOREST
- Train multiple decision trees on different random subsets of the data (bootstrapping).

- Optionally select a random subset of features at each split (feature bagging).

- Aggregate predictions using majority voting (for classification).

In [25]:
from collections import Counter

class RandomForest:
    def __init__(self, n_trees=5, max_depth=3, sample_size=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.sample_size = sample_size  # Size of data for bootstrapping
        self.trees = []

    def _bootstrap_sample(self, X, y):
        n_samples = self.sample_size or len(X)
        indices = np.random.choice(len(X), size=n_samples, replace=True)
        return X[indices], y[indices]

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree()
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree.fit(X_sample, y_sample, max_depth=self.max_depth)
            self.trees.append(tree)

    def _vote(self, predictions):
        count = Counter(predictions)
        return count.most_common(1)[0][0]

    def predict(self, X):
        # Collect predictions from all trees
        all_preds = np.array([tree.predict(X) for tree in self.trees])
        # Transpose so each row is a sample's predictions from all trees
        all_preds = all_preds.T
        # Majority vote for each sample
        final_preds = [self._vote(row) for row in all_preds]
        return final_preds


In [26]:
# Your previous dataset
X = np.array([
    [22, 25],
    [25, 30],
    [47, 60],
    [52, 80],
    [46, 50],
    [56, 90],
    [28, 40],
    [30, 60]
])
y = np.array([0, 0, 1, 1, 1, 1, 0, 0])

# Train the ensemble
forest = RandomForest(n_trees=5, max_depth=3)
forest.fit(X, y)

# Predict
predictions = forest.predict(X)
print("Ensemble Predictions:", predictions)

Ensemble Predictions: [np.int64(0), np.int64(0), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(0), np.int64(0)]


In [1]:
import numpy as np
from collections import Counter

# Reusing your DecisionTree with regression support
class TreeRegressor:
    def __init__(self, max_depth=3):
        self.max_depth = max_depth
        self.root = None

    def _mse(self, y):
        mean = np.mean(y)
        return np.mean((y - mean)**2)

    def _split(self, X, y, feature_index, threshold):
        left = X[:, feature_index] <= threshold
        return X[left], y[left], X[~left], y[~left]

    def _best_split(self, X, y):
        best_mse = float('inf')
        best_feat, best_thr = None, None
        for f in range(X.shape[1]):
            for thr in np.unique(X[:, f]):
                Xl, yl, Xr, yr = self._split(X, y, f, thr)
                if len(yl)==0 or len(yr)==0: continue
                mse = (len(yl)*self._mse(yl) + len(yr)*self._mse(yr)) / len(y)
                if mse < best_mse:
                    best_mse, best_feat, best_thr = mse, f, thr
        return best_feat, best_thr

    def _build(self, X, y, depth=0):
        if depth==self.max_depth or len(set(y))==1:
            return {'leaf': np.mean(y)}
        feat, thr = self._best_split(X, y)
        if feat is None:
            return {'leaf': np.mean(y)}
        Xl, yl, Xr, yr = self._split(X, y, feat, thr)
        return {
            'feat': feat,
            'thr': thr,
            'left': self._build(Xl, yl, depth+1),
            'right': self._build(Xr, yr, depth+1),
        }

    def fit(self, X, y):
        self.root = self._build(X, y)

    def _predict_one(self, node, x):
        if 'leaf' in node:
            return node['leaf']
        if x[node['feat']] <= node['thr']:
            return self._predict_one(node['left'], x)
        else:
            return self._predict_one(node['right'], x)

    def predict(self, X):
        return np.array([self._predict_one(self.root, x) for x in X])

class GradientBoostingRegressor:
    def __init__(self, n_estimators=5, learning_rate=0.1, max_depth=3):
        self.n_estimators = n_estimators
        self.lr = learning_rate
        self.max_depth = max_depth
        self.trees = []
        self.init_val = None

    def fit(self, X, y):
        # Start with mean prediction
        self.init_val = np.mean(y)
        pred = np.full_like(y, self.init_val, dtype=float)

        for m in range(self.n_estimators):
            
            residual = y - pred
            tree = TreeRegressor(max_depth=self.max_depth)
            tree.fit(X, residual)
            update = tree.predict(X)
            pred += self.lr * update
            self.trees.append(tree)

    def predict(self, X):
        pred = np.full((len(X),), self.init_val, dtype=float)
        for tree in self.trees:
            pred += self.lr * tree.predict(X)
        return pred


In [2]:
X = np.array([
    [6, 3],
    [8, 4],
    [5, 2],
    [7, 5],
    [6, 1]
])

y = np.array([4, 6, 3, 7, 2])

In [11]:
model = GradientBoostingRegressor(n_estimators=18, learning_rate=0.1, max_depth=3)
model.fit(X, y)

predictions = model.predict(X)
print(predictions)
rounded_preds = [int(round(p, 2)) for p in predictions]
print(rounded_preds)

[4.06003785 5.75984858 3.21013249 6.60975395 2.36022712]
[4, 5, 3, 6, 2]
